# Homework Assignment: Data Aggregation with Pandas


## Introduction
In this assignment, you will apply data aggregation techniques using the Pandas library. You will perform groupings and apply window functions to a sales dataset.



## Objectives
- Practice using the `groupby` function for data aggregation.
- Understand and apply rolling window functions.
- Explore expanding window functions for cumulative statistics.


**Load Data**: `sales_region_hw.csv`

In [35]:
import pandas as pd
sales_df = pd.read_csv('sales_region_hw.csv')

In [36]:
sales_df.head()

,Date,Region,Category,Sales,Quantity
0,2021-02-14,East,Furniture,715,33
1,2021-02-17,South,Electronics,59,15
2,2021-04-28,West,Furniture,955,62
3,2021-03-06,North,Furniture,353,88
4,2021-03-09,South,Electronics,579,33


In [37]:
# Total of 500 entries, no missing data. data type is valid for the columns
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      500 non-null    object
 1   Region    500 non-null    object
 2   Category  500 non-null    object
 3   Sales     500 non-null    int64 
 4   Quantity  500 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 19.7+ KB


In [38]:
sales_df.describe()

,Sales,Quantity
count,500.000000,500.000000
mean,490.934000,50.952000
std,280.336231,28.596665
min,10.000000,1.000000
25%,251.000000,25.750000
50%,476.000000,50.000000
75%,725.500000,76.000000
max,995.000000,99.000000


### Task 1: Group By Region and Category
- Perform the following operations and answer the questions below:

In [39]:
# 1. Group the data by 'Region' and 'Category'. What is the total sales amount for each group?
# regional_sales will display all 4 regions and the total sales amount for each group
region_sales = sales_df.groupby('Region')['Sales'].sum()
region_sales

,Sales
Region,
East,63757
North,61575
South,60250
West,59885


In [40]:
# 2. Which 'Region' and 'Category' combination has the highest average quantity sold?
region_category_avg = sales_df.groupby(['Region', 'Category'])['Quantity'].mean()
region_category_avg

Region  Category   
East    Clothing       46.162162
        Electronics    55.000000
        Furniture      53.583333
        Groceries      49.380952
North   Clothing       63.935484
        Electronics    57.148148
        Furniture      49.447368
        Groceries      44.482759
South   Clothing       52.793103
        Electronics    46.102564
        Furniture      56.151515
        Groceries      46.357143
West    Clothing       52.142857
        Electronics    45.235294
        Furniture      50.083333
        Groceries      51.441176
Name: Quantity, dtype: float64

In [41]:
# adding .idmax() at the end will provide the 'Region' and 'Category' that has the highest average
region_category_avg.idxmax()

('North', 'Clothing')

In [42]:
# 3. How many unique 'Category' entries are there for each 'Region'?
# using nunique() will describe the unique entries in "Category' for each 'Region'. The answer is 4 in each region
region_category_unique = sales_df.groupby('Region')['Category'].nunique()

In [43]:
region_category_unique

,Category
Region,
East,4
North,4
South,4
West,4


In [44]:
# 4. For each 'Region', what is the maximum sales value for 'Clothing'?
# ['Category'] == 'Clothing' is choosing specifically the column to provide a maximum sales value in each region.
sales_df[sales_df['Category'] == 'Clothing'].groupby('Region')['Sales'].max()


,Sales
Region,
East,970
North,994
South,995
West,961


In [45]:
# 5. Calculate the total 'Quantity' for each 'Category' across all 'Regions'. Which 'Category' has the highest total quantity?
# groupby the 'Category' by 'Quantity's sum to determine that Electronics is the highest total quantity with 6529 in 'Category'
region_category_quantity = sales_df.groupby('Category')['Quantity'].sum()

In [46]:
region_category_quantity

,Quantity
Category,
Clothing,6316
Electronics,6529
Furniture,6220
Groceries,6411


In [47]:
region_category_quantity.idxmax()

'Electronics'

### Task 2: Rolling Window Function
- Perform the following operations and answer the questions below:

In [48]:
# Perform the following operations and answer the questions below:
# 1. Calculate a 7-day rolling average of 'Sales'. On which date does the East region reach its highest 7-day rolling average of sales?

 # Instance 0 is restarting the 7-day roll, seeing as 'Sales' and '7_day_rolling_sales_avg' are the same
 # By creating a new column '7_day_rolling_sales_avg' I then put for the entries the rolling syntax to enter the mean within the past 7 days that moves along with the date
sales_df['7_day_rolling_sales_avg']= sales_df.groupby('Region')['Sales'].rolling(window=7, min_periods=1).mean().reset_index(drop=True)

In [49]:
sales_df.head(10)

,Date,Region,Category,Sales,Quantity,7_day_rolling_sales_avg
0,2021-02-14,East,Furniture,715,33,715.000000
1,2021-02-17,South,Electronics,59,15,722.500000
2,2021-04-28,West,Furniture,955,62,549.666667
3,2021-03-06,North,Furniture,353,88,434.500000
4,2021-03-09,South,Electronics,579,33,364.400000
5,2021-03-09,East,Furniture,730,57,450.500000
6,2021-04-14,North,Furniture,51,29,519.428571
7,2021-01-10,North,Furniture,484,44,465.000000
8,2021-03-25,North,Groceries,762,98,391.142857
9,2021-01-22,East,Furniture,204,51,427.714286


In [50]:
# To narrow down the data even more to focus on specifically the 'East'
east_region = sales_df[sales_df['Region'] == 'East']

In [51]:
# determine the max rolling avg date, specifically for the east_region displaying the 'Date'
east_region.head().sort_values(by='7_day_rolling_sales_avg', ascending=False)


,Date,Region,Category,Sales,Quantity,7_day_rolling_sales_avg
0,2021-02-14,East,Furniture,715,33,715.000000
18,2021-03-07,East,Electronics,89,94,528.142857
19,2021-04-13,East,Groceries,84,23,460.571429
5,2021-03-09,East,Furniture,730,57,450.500000
9,2021-01-22,East,Furniture,204,51,427.714286


In [52]:
max_rolling_avg_date = east_region.loc[east_region['7_day_rolling_sales_avg'] == east_region['7_day_rolling_sales_avg'].max()]['Date']


In [53]:
 # on 2021-03-09 the East region reached its highest of 7-day rolling average of sales with 528.142857, instance 18.
max_rolling_avg_date

,Date
0,2021-02-14


In [54]:
# 2. What is the overall average of the 7-day rolling sales amounts for each region?
# using the column '7_day_rolling_sales_avg' and the .mean() syntax, you are able to determine the overall average of the 7-day rolling sales for each region.
sales_df.groupby('Region')['7_day_rolling_sales_avg'].mean()

,7_day_rolling_sales_avg
Region,
East,500.035714
North,501.299371
South,485.847804
West,483.728614


In [ ]:
!jupyter nbconvert --to html "PASTE_THE_COPIED_FILE_PATH_HERE"